In [ ]:
from scipy.interpolate import CubicSpline
from sklearn.model_selection import cross_val_score
import numpy as np

# 큐빅 스플라인을 기반으로한 피쳐 선택 방법을 만들려면, 
# 먼저 각 피쳐에 대해 큐빅 스플라인 회귀를 수행하고, 
# 그 후 해당 회귀 모델의 잔차를 최소화하는 피쳐들을 선택하게 됨.
# 큐빅 스플라인은 데이터의 복잡한 패턴을 설명할 수 있는 비선형 회귀 모델 중 하나.
# 이를 활용하여 피쳐 선택을 수행하려면 각 피쳐에 대한 큐빅 스플라인의 적합도를 평가하고, 
# 잔차가 작은 피쳐들을 선택
# 클래스는 주어진 회귀 모델과 교차 검증 횟수를 입력으로 받고 
# fit_transform 메서드는 여러 임계값에 대해 교차 검증을 수행하여
# 최적의 임계값을 자동으로 찾고, 그 값을 사용하여 피쳐를 선택

class CubicSplineFeatureSelection:
    def __init__(self, threshold):
        self.threshold = threshold

    def fit_transform(self, X, y):
        residuals = []
        for feature in X.columns:
            x_data = X[feature]
            cs = CubicSpline(x_data.values, y.values)
            y_pred = cs(x_data)
            residual = np.mean((y - y_pred) ** 2)
            residuals.append(residual)

        # 피쳐 선택 기준 설정
        selected_features = [feature for idx, feature in enumerate(X.columns) if residuals[idx] < self.threshold]
        return X[selected_features]

class AutoThresholdCubicSplineFeatureSelection:
    def __init__(self, model, cv=5):
        self.model = model
        self.cv = cv

    def fit_transform(self, X, y):
        best_threshold = None
        best_score = float('inf')
        selector = CubicSplineFeatureSelection(threshold=0)  # 초기 클래스
        
        residuals = []
        for feature in X.columns:
            x_data = X[feature]
            cs = CubicSpline(x_data.values, y.values)
            y_pred = cs(x_data)
            residual = np.mean((y - y_pred) ** 2)
            residuals.append(residual)
        
        for threshold in np.linspace(min(residuals), max(residuals), 100):
            
            selector.threshold = threshold
            
            X_selected = selector.fit_transform(X, y)
            
            score = -np.mean(cross_val_score(self.model, X_selected, y, cv=self.cv, scoring="neg_mean_squared_error"))
            
            if score < best_score:
                best_score = score
                best_threshold = threshold
        
        selector.threshold = best_threshold
        return selector.fit_transform(X, y)